In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [2]:
spark = (
    SparkSession.builder.appName("preprocessing of taxi data")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "15g")
    .getOrCreate()
)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/08 22:18:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/08/08 22:18:30 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
sdf = spark.read.parquet("../data/curated/preprocess_result1")
sdf.show()
print(sdf.count())

+--------------------+---------------------+------------+------------+--------------+
|tpep_pickup_datetime|tpep_dropoff_datetime|PULocationID|DOLocationID|duration (sec)|
+--------------------+---------------------+------------+------------+--------------+
| 2016-03-01 00:53:35|  2016-03-01 00:57:11|         226|         226|           216|
| 2016-03-01 00:15:13|  2016-03-01 00:15:26|         145|         145|            13|
| 2016-03-01 00:15:46|  2016-03-01 00:16:00|         145|         145|            14|
| 2016-03-01 00:16:17|  2016-03-01 00:16:23|         145|         145|             6|
| 2016-03-01 00:16:45|  2016-03-01 00:18:14|         145|         145|            89|
| 2016-03-01 00:18:42|  2016-03-01 00:19:12|         145|         145|            30|
| 2016-03-01 00:19:34|  2016-03-01 00:19:53|         145|         145|            19|
| 2016-03-01 00:20:08|  2016-03-01 00:20:42|         145|         145|            34|
| 2016-03-01 00:20:57|  2016-03-01 00:21:14|         1